1. Enable data augmentation, and precompute=True
2. Use lr_find() to find highest learning rate where loss is still clearly improving
3. Train last layer from precomputed activations for 1-2 epochs
4. Train last layer with data augmentation (i.e. precompute=False) for 2-3 epochs with cycle_len=1
5. Unfreeze all layers
6. Set earlier layers to 3x-10x lower learning rate than next higher layer
7. Use lr_find() again
8. Train full network with cycle_mult=2 until over-fitting


# Pre-requistes

In [1]:
# Put these at the top of every notebook, to get automatic reloading and inline plotting
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
# This file contains all the main external libs we'll use
from fastai.imports import *

ModuleNotFoundError: No module named 'bcolz'

In [ ]:
from fastai.transforms import *
from fastai.conv_learner import *
from fastai.model import *
from fastai.dataset import *
from fastai.sgdr import *
from fastai.plots import *

In [ ]:
PATH = "/home/paperspace/data/dogscats/"

## Get data

### Directory based structure

In [3]:
!ls {PATH}

ls: {PATH}: No such file or directory


In [ ]:
!ls {PATH}valid]

In [ ]:
files = !ls {PATH}valid/cats | head
files

In [4]:
img = plt.imread(f'{PATH}valid/cats/{files[0]}')
plt.imshow(img)

NameError: name 'plt' is not defined

In [ ]:
# Image data (pixel_col, pixel_row, RGB)
img.shape 

In [ ]:
def get_tfms(size): return tfms_from_model(f_model, size)

In [ ]:
def get_aug_tfms(size): return tfms_from_model(f_model, size, aug_tfms=transforms_side_on, max_zoom=1.1)

In [ ]:
def get_data_from_csv(sz, tfms=gettfms(sz)):
    return ImageClassifierData.from_csv(PATH, 'train-jpg', label_csv, tfms=tfms,
                    suffix='.jpg', val_idxs=val_idxs, test_name='test-jpg')

In [ ]:
def get_data_from_dir(sz, tfms=gettfms(sz)):
    return ImageClassifierData.from_paths(PATH, tfms=tfms_from_model(arch, sz), test_name='test1')

# Enable data augmentation, and precompute=True

## Initial model setup

In [ ]:
arch=resnet34
size=224

In [5]:
data = get_data_from_dir(size)

NameError: name 'get_data_from_dir' is not defined

In [ ]:
learn = ConvLearner.pretrained(f_model, data, metrics=metrics)

# Use lr_find() to find highest learning rate where loss is still clearly improving

In [ ]:
discovered_learning_rate = learn.lr_find()
learn.sched.plot_lr()
learn.sched.plot()

In [ ]:
learning_rate = 0.01

# Train last layer from precomputed activations for 1-2 epochs

In [ ]:
learn.fit(learning_rate, 3)

## Analyse output

# Train last layer with data augmentation (i.e. precompute=False) for 2-3 epochs with cycle_len=1

In [ ]:
data = get_data_from_dir(size, get_aug_tfms(size))

In [ ]:
learn = ConvLearner.pretrained(f_model, data, metrics=metrics)

In [ ]:
learn.precompute=False

In [ ]:
learn.fit(learning_rate, 3, cycle_len=1)

In [6]:
learn.save(f'last_layer_{size}')

NameError: name 'learn' is not defined

# Train full network with cycle_mult=2 until over-fitting

### Unfreeze all layers

In [ ]:
learn.unfreeze()

### Find learning rate again

In [ ]:
learn = ConvLearner.pretrained(f_model, data, metrics=metrics)

In [ ]:
new_learning_rate = learn.lr_find()
plot.sched.plot_lr()
plot.sched.plot()

### Set earlier layers to 3x-10x lower learning rate than next higher layer

In [ ]:
new_learning_rate = 0.01
learning_rates = np.array([lr/9,lr/3,new_learning_rate])

### Train entire network

In [7]:
learn.fit(learning_rate, 3, cycle_len=1, cycle_mult=2)

NameError: name 'learn' is not defined

In [ ]:
learn.save(f'entire_network_{size}')

# Check accuracy

## Validation set

In [ ]:
log_preds, y = learn.TTA()
probs = np.mean(np.exp(log_preds), 0)
accuracy(probs, y)

## Test set

In [8]:
test_log_preds, test_y = learn.TTA(True)
test_probs = np.mean(np.exp(test_log_preds), 0)
print(accuracy(test_probs, test_y))

print(test_probs.shape)
print(test_probs[:4])

NameError: name 'learn' is not defined

# Submit to kaggle (optional)

In [ ]:
isDog = test_probs[:,1]
print(isDog.shape)
print(len(isDog))
print(isDog[:5])


In [103]:
test_ds = learn.data.test_ds
print(test_ds)
test_dl = learn.data.test_dl
test_filenames = test_dl.dataset.fnames
print(test_filenames[:5])
print(test_dl.dataset.y.max())

['test1/9885.jpg', 'test1/2701.jpg', 'test1/11231.jpg', 'test1/2964.jpg', 'test1/10651.jpg']
0.0


In [ ]:
fileIds = np.array([int(f[6:f.find('.')]) for f in test_filenames])
fileIds[:5]

In [ ]:
submission = np.stack([fileIds,isDog], axis=1)
submission[:5]

In [108]:
!pwd

/home/paperspace/fastai/courses/dl1


In [111]:
submission_file_name = 'submission1.csv'
# changed the decimal format from .5 to .10 and NO clip and moved into top 8% on kaggle (from 10%)
np.savetxt(submission_file_name, submission, fmt='%d,%.10f', header='id,label', comments='')

In [112]:
from IPython.display import FileLink
FileLink(submission_file_name)

/home/paperspace/fastai/courses/dl1/submission1.csv